# Import Libraries

In [1]:
import optuna
import os
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [2]:
def read_file(*args,file_type="csv")->pd.DataFrame:
    """:parameter input directories in sequence
        :return DataFrame"""
    
    path=os.path.join(*args)
    if file_type == "tsv":
        return pd.read_csv(path, sep="\t")
    return pd.read_csv(path)

# Experiments

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [4]:
mlflow.set_experiment("DifferentModelsExperiments2")

2024/10/19 13:06:27 INFO mlflow.tracking.fluent: Experiment with name 'DifferentModelsExperiments2' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/spynom/mlruns/14', creation_time=1729323387786, experiment_id='14', last_update_time=1729323387786, lifecycle_stage='active', name='DifferentModelsExperiments2', tags={}>

In [5]:
df=read_file("..","data","processed","cleaned.csv").dropna(how="any").drop_duplicates()

X=df.comment
y=df.category+1

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
vectorizer.fit(X)  # Fit on all vocab data
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)  # Transform test data

In [6]:

class_weights = compute_class_weight('balanced', classes=np.array([0, 1, 2]), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

In [10]:
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_BalanceWeight_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        train_y_pred=model.predict(X_train)
        test_y_pred = model.predict(X_test)

        # Log accuracy
        train_accuracy = accuracy_score(y_train,train_y_pred)
        test_accuracy = accuracy_score(y_test, test_y_pred)
        
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("test_accuracy", test_accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, test_y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


In [11]:
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)  # Tuning the number of neighbors
    p = trial.suggest_categorical('p', [1, 2])  # Tuning the distance metric (1 for Manhattan, 2 for Euclidean)

    # KNeighborsClassifier setup
    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for KNN, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    # Log the best model with MLflow, passing the algo_name as "KNN"
    log_mlflow("KNN", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for KNN
run_optuna_experiment()

[I 2024-10-19 13:14:14,565] A new study created in memory with name: no-name-a0d54948-1137-4aac-8552-055f6a3b2410
[I 2024-10-19 13:14:21,090] Trial 0 finished with value: 0.36258278145695366 and parameters: {'n_neighbors': 11, 'p': 2}. Best is trial 0 with value: 0.36258278145695366.
[I 2024-10-19 13:14:27,558] Trial 1 finished with value: 0.3511313465783664 and parameters: {'n_neighbors': 20, 'p': 2}. Best is trial 0 with value: 0.36258278145695366.
[I 2024-10-19 13:14:42,368] Trial 2 finished with value: 0.37375827814569534 and parameters: {'n_neighbors': 5, 'p': 1}. Best is trial 2 with value: 0.37375827814569534.
[I 2024-10-19 13:14:50,154] Trial 3 finished with value: 0.36258278145695366 and parameters: {'n_neighbors': 11, 'p': 2}. Best is trial 2 with value: 0.37375827814569534.
[I 2024-10-19 13:14:57,400] Trial 4 finished with value: 0.35471854304635764 and parameters: {'n_neighbors': 17, 'p': 2}. Best is trial 2 with value: 0.37375827814569534.
[I 2024-10-19 13:15:05,054] Trial